Get results by criteria

In [1]:
from glob import glob
import os
import json
import numpy as np
import yaml
import pandas as pd

def update_dict(d, key, value):
    if key not in d:
        d[key] = [value]
    else:
        d[key].append(value)
    return d

output_file_name = 'quorum_output_field.yaml'
if os.path.exists(output_file_name):
    os.remove(output_file_name)

centers = {}
manufacturers = {}
strengths = {}
depths = {}
phases = {}

path_list = glob(r'quorum_output_2\fold_0\temp_allClasses\*.gz')
for path in path_list:
    filename = path.split('\\')[-1]
    phase = filename[11:13]
    name = filename.split('_')[0]
    df = pd.read_csv(os.path.join('custom_quorum_2', name, phase + '_info.csv'))
    center = df['Name'].iloc[0].split('-')[0]
    manufacturer = df['Manufacturer'].iloc[0]
    strength = df['Field Strength'].iloc[0]

    centers = update_dict(centers, str(center), filename)
    manufacturers = update_dict(manufacturers, str(manufacturer), filename)
    strengths = update_dict(strengths, str(strength), filename)
    phases = update_dict(phases, str(phase), filename)

with open(r'quorum_output_2\fold_0\temp_allClasses\summary.json', 'r') as fd_in:
    metric_file = json.load(fd_in)['results']['all']
    results_dict = {}
    for current_dict in [centers, manufacturers, strengths, phases]:
        for key in current_dict.keys():
            print(f'{key} = {len(current_dict[key])}')
            current_values = current_dict[key]
            list_of_dict = [x for x in metric_file if x['reference'].split('/')[-1] in current_values]
            mean_dice_list = []
            mean_hausdorff_list = []
            for data_dict in list_of_dict:
                mean_dice_list.append([data_dict['1']['Dice'], data_dict['2']['Dice'], data_dict['3']['Dice']])
                mean_hausdorff_list.append([data_dict['1']['Hausdorff Distance'], data_dict['2']['Hausdorff Distance'], data_dict['3']['Hausdorff Distance']])
            class_dice = np.stack(mean_dice_list, axis=0).mean(axis=0)
            class_hausdorff = np.stack(mean_hausdorff_list, axis=0).mean(axis=0)
            results_dict[key] = {'Hausdorff distance': class_hausdorff.tolist(), 
                                'Mean Hausdorff distance': float(class_hausdorff.mean()), 
                                'Dice score': class_dice.tolist(),
                                'Mean dice score': float(class_dice.mean())}

with open(output_file_name, 'w') as fd:
    yaml.safe_dump(results_dict, fd, default_flow_style=False)
    #for results_dict in results_dicts:
    #    for key in results_dict.keys():
    #        fd.write(key + ': ' + str(results_dict[key]) + '\n')
    #    fd.write('\n')


250001 = 10
250005 = 2
276004 = 2
276005 = 2
276008 = 2
348001 = 6
348002 = 6
348003 = 2
348004 = 8
348007 = 6
616003 = 4
616005 = 2
616010 = 2
703001 = 10
703003 = 2
703004 = 8
724002 = 2
724006 = 4
siemens = 62
philips = 4
ge = 14
1.5 = 66
3.0 = 14
ed = 40
es = 40


Get ED/ES volume

In [5]:
import nibabel as nib
import numpy as np

data = nib.load(r'quorum_output\validation_raw\patient003_ed.nii.gz')
zoom = data.header.get_zooms()
pixel_volume = np.prod(zoom)
print(pixel_volume)
arr = data.get_fdata()
nb_pixels = np.count_nonzero(arr == 1)
print(nb_pixels * pixel_volume)

13.234375
187504.625


In [2]:
%matplotlib qt

import nibabel as nib
import numpy as np
from scipy.ndimage import distance_transform_edt
from scipy.interpolate import interpn
import matplotlib.pyplot as plt

def bwperim(bw, n=4):
    """
    perim = bwperim(bw, n=4)
    Find the perimeter of objects in binary images.
    A pixel is part of an object perimeter if its value is one and there
    is at least one zero-valued pixel in its neighborhood.
    By default the neighborhood of a pixel is 4 nearest pixels, but
    if `n` is set to 8 the 8 nearest pixels will be considered.
    Parameters
    ----------
      bw : A black-and-white image
      n : Connectivity. Must be 4 or 8 (default: 8)
    Returns
    -------
      perim : A boolean image
    """

    if n not in (4,8):
        raise ValueError('mahotas.bwperim: n must be 4 or 8')
    rows,cols = bw.shape

    # Translate image by one pixel in all directions
    north = np.zeros((rows,cols))
    south = np.zeros((rows,cols))
    west = np.zeros((rows,cols))
    east = np.zeros((rows,cols))

    north[:-1,:] = bw[1:,:]
    south[1:,:]  = bw[:-1,:]
    west[:,:-1]  = bw[:,1:]
    east[:,1:]   = bw[:,:-1]
    idx = (north == bw) & \
          (south == bw) & \
          (west  == bw) & \
          (east  == bw)
    if n == 8:
        north_east = np.zeros((rows, cols))
        north_west = np.zeros((rows, cols))
        south_east = np.zeros((rows, cols))
        south_west = np.zeros((rows, cols))
        north_east[:-1, 1:]   = bw[1:, :-1]
        north_west[:-1, :-1] = bw[1:, 1:]
        south_east[1:, 1:]     = bw[:-1, :-1]
        south_west[1:, :-1]   = bw[:-1, 1:]
        idx &= (north_east == bw) & \
               (south_east == bw) & \
               (south_west == bw) & \
               (north_west == bw)
    return ~idx * bw

def signed_bwdist(im):
    '''
    Find perim and return masked image (signed/reversed)
    '''    
    perimeter = bwperim(im)

    distance_map = bwdist(perimeter)

    im = -distance_map*np.logical_not(im) + distance_map*im
    return im

def bwdist(im):
    '''
    Find distance map of image
    '''
    dist_im = distance_transform_edt(1-im)
    return dist_im

def interp_shape(arr, new_depth):
    '''
    Interpolate between two contours

    Input: top 
            [X,Y] - Image of top contour (mask)
           bottom
            [X,Y] - Image of bottom contour (mask)
           precision
             float  - % between the images to interpolate 
                Ex: num=0.5 - Interpolate the middle image between top and bottom image
    Output: out
            [X,Y] - Interpolated image at num (%) between top and bottom

    '''
    X, Y, Z = arr.shape

    distance_arr = []
    for i in range(Z):
        distance_arr.append(signed_bwdist(arr[:, :, i]))
    distance_arr = np.stack(distance_arr, axis=-1)

    x = np.arange(0, X)
    y = np.arange(0, Y)
    z = np.arange(0, Z)
    points = (x, y, z)

    stop = Z-1

    # create ndgrids
    grid = np.mgrid[:X, :Y, 0:stop:(new_depth * 1j)]
    xi = np.rollaxis(grid, 0, 4)
    xi = xi.reshape((X * Y * new_depth, 3))

    out = interpn(points, distance_arr, xi)
    out = out.reshape((X, Y, new_depth))

    # Threshold distmap to values above 0
    out = out > 0

    #fig, ax = plt.subplots(2, out.shape[-1])
    #for t in range(out.shape[-1]):
    #    if t < arr.shape[-1]:
    #        ax[0, t].imshow(arr[:, :, t], cmap='gray')
    #    ax[1, t].imshow(out[:, :, t], cmap='gray')
    #plt.show()

    #print(out.shape)

    return out


data = nib.load(r'data_saud_2\inference\patient030_ed.nii.gz')
arr = data.get_fdata()
arr = arr == 1

X, Y, Z = arr.shape
#print(arr.shape)
# Run interpolation
out = interp_shape(arr, Z+1)
#print(out.shape)
#fig, ax = plt.subplots(1, Z+1)
#for i in range(Z+1):
#    ax[i].imshow(out[:, :, i], cmap='gray')
#plt.show()

Get only patient in test set

In [3]:
from glob import glob
import pandas as pd

def update_dict(d, key, value):
    if key not in d:
        d[key] = [value]
    else:
        d[key].append(value)
    return d


cut_prediction_path = r'quorum_output_2\fold_0\temp_allClasses'
all_prediction_path = r'data_saud_2\inference'

data_dict_cut = {}
data_dict_all = {}

path_list = glob(os.path.join(cut_prediction_path, '*.gz'))
cut_prediction_names = []
for path in path_list:
    filename = path.split('\\')[-1]
    phase = filename[11:13]
    name = filename.split('_')[0]
    df = pd.read_csv(os.path.join('custom_quorum_2', name, phase + '_info.csv'))
    actual_name = df['Name'].to_numpy()[0]
    spacing = (df['Space Between Slices'] - df['Slice Thickness']).to_numpy()[0]
    cut_prediction_names.append(actual_name)
    if phase == 'ed':
        update_dict(data_dict_cut, 'ed', (path, spacing))
    elif phase == 'es':
        update_dict(data_dict_cut, 'es', (path, spacing))

csv_list = glob(os.path.join(all_prediction_path, '*.csv'))
for csv_path in csv_list:
    filename = csv_path.split('\\')[-1]
    phase = filename[11:13]
    df = pd.read_csv(csv_path)
    actual_name = df['Name'].to_numpy()[0]
    if actual_name in cut_prediction_names:
        spacing = (df['Space Between Slices'] - df['Slice Thickness']).to_numpy()[0]
        path = csv_path[:-4] + '.nii.gz'
        if phase == 'ed':
            update_dict(data_dict_all, 'ed', (path, spacing))
        elif phase == 'es':
            update_dict(data_dict_all, 'es', (path, spacing))

In [4]:
%matplotlib qt

import nibabel as nib
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import csv

#path_list = glob(r'quorum_output_2\temp_allClasses\*.gz')

for data_dict, output_file_name in zip([data_dict_cut, data_dict_all], ['quorum_output_volume_cut.csv', 'quorum_output_volume_all.csv']):
    if os.path.exists(output_file_name):
        os.remove(output_file_name)
    results = []
    for i, class_name in enumerate(tqdm(['RV', 'MYO', 'LV']), 1):
        for phase in data_dict.keys():
            list_of_tuple = data_dict[phase]
            for path, spacing in list_of_tuple:
                data = nib.load(path)
                zoom = list(data.header.get_zooms())
                arr = data.get_fdata()
                pixel_size = np.prod(zoom)

                new_depth = round((spacing * (arr.shape[-1] - 1)) / zoom[-1]) + arr.shape[-1]
                #print(arr.shape[-1])
                #print(new_depth)
                #print('******************************')

                class_nb_pixels = arr == i

                volume_no_interp = (pixel_size * class_nb_pixels.sum()) / 1000
                if class_name == 'MYO':
                    volume_no_interp = volume_no_interp * 1.055

                if new_depth > arr.shape[-1]:
                    arr_interpolated = interp_shape(class_nb_pixels, new_depth)
                    volume_interp = (pixel_size * arr_interpolated.sum()) / 1000
                    if class_name == 'MYO':
                        volume_interp = volume_interp * 1.055
                else:
                    volume_interp = volume_no_interp

                results.append({'Phase': phase, 'Class': class_name, 'Volume': volume_no_interp, 'Interpolated_volume': volume_interp})
            
        #if class_name != 'MYO':
        #    results[class_name].update({
        #        'Fraction d\'ejection': {'no_interpolation': ((results[class_name]['ed']['volume'] - results[class_name]['es']['volume']) / results[class_name]['ed']['volume']) * 100,
        #                                'interpolated': ((results[class_name]['ed']['interpolated_volume'] - results[class_name]['es']['interpolated_volume']) / results[class_name]['ed']['interpolated_volume']) * 100},
        #        'Volume d\'ejection': {'no_interpolation': (results[class_name]['ed']['volume'] - results[class_name]['es']['volume']),
        #                                'interpolated': (results[class_name]['ed']['interpolated_volume'] - results[class_name]['es']['interpolated_volume'])}
        #                                })          

    with open(output_file_name, 'w') as fd:
        writer = csv.DictWriter(fd, fieldnames=list(results[0].keys()))
        writer.writeheader() 
        writer.writerows(results) 

100%|██████████| 3/3 [00:08<00:00,  2.73s/it]


In [5]:
import json
import os
import csv

folders = [r'Quorum_trainings\Baseline', r'Quorum_trainings\no_trans', r'Quorum_trainings\no_sfb', r'Quorum_trainings\no_ds']
results_list = []
for folder in folders:
    with open(os.path.join(folder, r'fold_0\temp_allClasses\summary.json')) as fd_json:
        data = json.load(fd_json)
        results = data['results']['all']
        for res in results:
            rv_dice = res['1']['Dice']
            myo_dice = res['2']['Dice']
            lv_dice = res['3']['Dice']
            results_list.append({'Method': folder.split('\\')[-1], 'RV': rv_dice, 'MYO': myo_dice, 'LV': lv_dice, 'Mean': (rv_dice + myo_dice + lv_dice) / 3})

with open(os.path.join('Quorum_trainings', 'jmp.csv'), 'w') as fd_csv:
    writer = csv.DictWriter(fd_csv, fieldnames=list(results_list[0].keys()))
    writer.writeheader() 
    writer.writerows(results_list) 
        

In [27]:
class ConfusionMatrix:

    def __init__(self, test=None, reference=None):

        self.tp = None
        self.fp = None
        self.tn = None
        self.fn = None
        self.size = None
        self.reference_empty = None
        self.reference_full = None
        self.test_empty = None
        self.test_full = None
        self.set_reference(reference)
        self.set_test(test)

    def set_test(self, test):

        self.test = test
        self.reset()

    def set_reference(self, reference):

        self.reference = reference
        self.reset()

    def reset(self):

        self.tp = None
        self.fp = None
        self.tn = None
        self.fn = None
        self.size = None
        self.test_empty = None
        self.test_full = None
        self.reference_empty = None
        self.reference_full = None

    def compute(self):

        if self.test is None or self.reference is None:
            raise ValueError("'test' and 'reference' must both be set to compute confusion matrix.")

        assert_shape(self.test, self.reference)

        self.tp = int(((self.test != 0) * (self.reference != 0)).sum())
        self.fp = int(((self.test != 0) * (self.reference == 0)).sum())
        self.tn = int(((self.test == 0) * (self.reference == 0)).sum())
        self.fn = int(((self.test == 0) * (self.reference != 0)).sum())
        self.size = int(np.prod(self.reference.shape, dtype=np.int64))
        self.test_empty = not np.any(self.test)
        self.test_full = np.all(self.test)
        self.reference_empty = not np.any(self.reference)
        self.reference_full = np.all(self.reference)

    def get_matrix(self):

        for entry in (self.tp, self.fp, self.tn, self.fn):
            if entry is None:
                self.compute()
                break

        return self.tp, self.fp, self.tn, self.fn

    def get_size(self):

        if self.size is None:
            self.compute()
        return self.size

    def get_existence(self):

        for case in (self.test_empty, self.test_full, self.reference_empty, self.reference_full):
            if case is None:
                self.compute()
                break

        return self.test_empty, self.test_full, self.reference_empty, self.reference_full


def dice(test=None, reference=None, confusion_matrix=None, nan_for_nonexisting=True, **kwargs):
    """2TP / (2TP + FP + FN)"""

    if confusion_matrix is None:
        confusion_matrix = ConfusionMatrix(test, reference)

    tp, fp, tn, fn = confusion_matrix.get_matrix()
    test_empty, test_full, reference_empty, reference_full = confusion_matrix.get_existence()

    if test_empty and reference_empty:
        if nan_for_nonexisting:
            return float("NaN")
        else:
            return 0.

    return float(2. * tp / (2 * tp + fp + fn))

Get dice per depth level

In [43]:
from evaluation.metrics import dice
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from glob import glob
import math

def update_dict(d, key, value):
    if key not in d:
        d[key] = [value]
    else:
        d[key].append(value)
    return d

path_list = glob(r'quorum_output_2\fold_0\temp_allClasses\*.gz')
path_list_names = [x.split('\\')[-1][:13] for x in path_list]
path_list_gt = glob(r'custom_quorum_2\**\*_gt.nii.gz', recursive=True)
path_list_gt = [x for x in path_list_gt if x.split('\\')[-1][:13] in path_list_names]
assert len(path_list_gt) == len(path_list)

path_list = sorted(path_list, key=lambda x:x.split('\\')[-1])
path_list_gt = sorted(path_list_gt, key=lambda x:x.split('\\')[-1])

out_dict = {}
scores = []
for path, path_gt in zip(path_list, path_list_gt):
    data = nib.load(path)
    arr = data.get_fdata()

    data_gt = nib.load(path_gt)
    arr_gt = data_gt.get_fdata()

    assert arr.shape == arr_gt.shape
    patient_scores = []
    for i in range(arr.shape[-1]):
        #fig, ax = plt.subplots(1, 2)
        #ax[0].imshow(arr[:, :, i], cmap='gray')
        #ax[1].imshow(arr_gt[:, :, i], cmap='gray')
        #plt.show()
        #plt.waitforbuttonpress()
        #plt.close(fig)

        current_pred = arr[:, :, i]
        current_gt = arr_gt[:, :, i]

        #current_pred = arr
        #current_gt = arr_gt

        class_score = []
        for j in range(1, 4):
            current_class_pred = current_pred == j
            current_gt_pred = current_gt == j
            score = dice(current_class_pred, current_gt_pred)
            class_score.append(score)
        out_dict = update_dict(out_dict, key=i/arr.shape[-1], value=np.nanmean(np.array(class_score)))
        patient_scores.append(np.array(class_score))
    patient_class_score = np.stack(patient_scores, axis=0)
    patient_class_score = np.nanmean(patient_class_score, axis=0)
    scores.append(patient_class_score)
scores = np.stack(scores, 0)
scores = np.nanmean(scores, axis=0)

x = []
y = []
for key in out_dict.keys():
    x.append(key)
    y.append(np.array(out_dict[key]).mean())

print(np.array(scores).mean())

fig, ax = plt.subplots(1, 1)
ax.scatter(np.array(x), np.array(y))
ax.set(xlabel='Depth as percent of volume', ylabel='Dice score')
ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))

[0.85727813 0.91870862 0.88975411]
[0.85125525 0.9503675  0.92224149]
[0.81496491 0.88242957 0.88595113]
[0.865601   0.89990146 0.89230452]
[0.85498399 0.84753379 0.86183659]
[0.80896762 0.91509044 0.83652693]
[0.94568635 0.86088831 0.9479568 ]
[0.92407612 0.90031627 0.90839191]
[0.94200341 0.89772142 0.9633841 ]
[0.88622391 0.93359071 0.94075803]
[0.93152901 0.91623481 0.95633951]
[0.82610928 0.9333228  0.93242686]
[0.9255292  0.89393533 0.88108325]
[0.9458467  0.87076002 0.88045608]
[0.91147253 0.89839016 0.95945507]
[0.74979777 0.89845404 0.94584293]
[0.87038465 0.83833018 0.86606815]
[0.56653339 0.78370152 0.78536067]
[0.83022049 0.90489216 0.93878273]
[0.84307281 0.8781236  0.94790739]
[0.94290595 0.91856772 0.96703108]
[0.92860577 0.93235305 0.91357425]
[0.88386899 0.89478284 0.95482796]
[0.88520016 0.91129737 0.9507714 ]
[0.84851985 0.92478445 0.88103115]
[0.71604308 0.90891398 0.84393   ]
[0.76101985 0.86485348 0.86941596]
[0.45169637 0.6704366  0.65358929]
[0.94472996 0.884971